In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

/home/sbsatter/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
mnist = input_data.read_data_sets('MNIST_data/', one_hot=True)
sess = tf.InteractiveSession()

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
x = tf.placeholder(tf.float32, shape=(None, 784), name='x')
y_ = tf.placeholder(tf.float32, shape=(None, 10), name='y_')

x_image = tf.reshape(x, [-1, 28, 28, 1], name='x_image')
print(x_image.shape)

(?, 28, 28, 1)


In [4]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)
def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')
def maxpool2d(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1,2,2,1], padding='VALID')

In [5]:
W1 = weight_variable([5,5,1,32])
b1 = bias_variable([32])

conv1 = tf.nn.relu(conv2d(x_image, W1) + b1) 
maxpool1 = maxpool2d(conv1) # fig

W2 = weight_variable([5,5,32,64])
b2 = bias_variable([64])

conv2 = tf.nn.relu(conv2d(maxpool1, W2) + b2) 
maxpool2 = maxpool2d(conv2)

W3 = weight_variable([7*7*64, 1024])
b3 = weight_variable([1024])

maxpool2_flat = tf.reshape(maxpool2, [-1, 7*7*64])
fc1 = tf.nn.relu(tf.matmul(maxpool2_flat, W3) + b3)

keep_prob = tf.placeholder(tf.float32)
dropout = tf.nn.dropout(fc1, keep_prob)

In [6]:
W4 = weight_variable([1024,10])
b4 = bias_variable([10])

y_conv = tf.matmul(dropout, W4) + b4


cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

sess.run(tf.global_variables_initializer())

import time

start = time.time()
end = time.time()

num_steps = 3000
display_every = 200

for i in range(num_steps):
    batch = mnist.train.next_batch(64)
#     for j in range(len(batch)):
#         print('batch[{0}] => {1}\n'.format(j, batch[j].shape))
    train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})
    
    if i % display_every == 0:
        train_accuracy = accuracy.eval(feed_dict={x: batch[0], y_: batch[1], keep_prob: 1})
        end = time.time()
        
        print('step {0}  elapsed time {1:.2f} seconds, training accuracy: {2:.3f}'.format(i, end - start, train_accuracy))
end = time.time()
print('Training time for {0} batches: {1:.2f} seconds'.format(i, end - start))
print('Test Accuracy: {0}', accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1}))
sess.close()

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

step 0  elapsed time 0.68 seconds, training accuracy: 0.234
step 200  elapsed time 24.10 seconds, training accuracy: 0.891
step 400  elapsed time 50.16 seconds, training accuracy: 0.969
step 600  elapsed time 79.34 seconds, training accuracy: 0.906
step 800  elapsed time 104.76 seconds, training accuracy: 0.938
step 1000  elapsed time 134.50 seconds, training accuracy: 0.922
step 1200  elapsed time 160.48 seconds, training accuracy: 0.969
step 1400  elapsed time 189.93 seconds, training accuracy: 0.984
step 1600  elapsed time 217.23 seconds, training accuracy: 0.984
step 1800  elapsed time 241.59 seconds, training accuracy: 1.000
step 2000  elapsed time 273.19 seconds, training accuracy: 0.938
step 2200  elapsed time 305.38 seconds, training accuracy: 0.984
step 2400  elapsed time 333.89 seconds, 

In [7]:
mnist.train.num_examples

55000

In [8]:
a = (mnist.train.next_batch(64))
print(a[0].shape)
print(a[1].shape)

(64, 784)
(64, 10)
